In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
cust = pd.read_csv("Customers.csv")
prod = pd.read_csv("Products.csv")
trans = pd.read_csv("Transactions.csv")

In [3]:
cust.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [4]:
trans.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [5]:
prod.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [6]:
merged_data = pd.merge(cust, trans, on='CustomerID', how='left')
denormalized_data = pd.merge(merged_data, prod, on='ProductID', how='left')

In [7]:
denormalized_data.head()

,CustomerID,CustomerName,Region,SignupDate,TransactionID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y
0,C0001,Lawrence Carroll,South America,2022-07-10,T00015,P054,2024-01-19 03:12:55,2.0,114.60,57.30,SoundWave Cookbook,Books,57.30
1,C0001,Lawrence Carroll,South America,2022-07-10,T00932,P022,2024-09-17 09:01:18,3.0,412.62,137.54,HomeSense Wall Art,Home Decor,137.54
2,C0001,Lawrence Carroll,South America,2022-07-10,T00085,P096,2024-04-08 00:01:00,2.0,614.94,307.47,SoundWave Headphones,Electronics,307.47
3,C0001,Lawrence Carroll,South America,2022-07-10,T00445,P083,2024-05-07 03:11:44,2.0,911.44,455.72,ActiveWear Smartwatch,Electronics,455.72
4,C0001,Lawrence Carroll,South America,2022-07-10,T00436,P029,2024-11-02 17:04:16,3.0,1300.92,433.64,TechPro Headphones,Electronics,433.64


In [8]:

customer_transactions = denormalized_data.groupby('CustomerID')['Category'].apply(list).reset_index()

customer_transactions = customer_transactions.rename(columns={'Category': 'Transactions'})

final_data = pd.DataFrame(customer_transactions.Transactions.values.tolist(), index=customer_transactions.CustomerID).add_prefix('Trans')

final_data = final_data.reset_index()

final_data.head()


,CustomerID,Trans0,Trans1,Trans2,Trans3,Trans4,Trans5,Trans6,Trans7,Trans8,Trans9,Trans10
0,C0001,Books,Home Decor,Electronics,Electronics,Electronics,None,None,None,None,None,None
1,C0002,Home Decor,Home Decor,Clothing,Clothing,None,None,None,None,None,None,None
2,C0003,Home Decor,Home Decor,Clothing,Electronics,None,None,None,None,None,None,None
3,C0004,Books,Home Decor,Home Decor,Home Decor,Books,Books,Electronics,Electronics,None,None,None
4,C0005,Home Decor,Electronics,Electronics,None,None,None,None,None,None,None,None


In [9]:
def count_transactions(row):
    counts = {}
    for col in row[1:]:
        if col != 'None':
            counts[col] = counts.get(col, 0) + 1
    return counts

count_data = pd.DataFrame(final_data.apply(count_transactions, axis=1).tolist()).fillna(0).astype(int)

print(count_data)
final_data = pd.concat([final_data['CustomerID'], count_data], axis=1)

final_data.head()

     Books  Home Decor  Electronics  None  Clothing  NaN
0        1           1            3     6         0    0
1        0           2            0     7         2    0
2        0           2            1     7         1    0
3        3           3            2     3         0    0
4        0           1            2     8         0    0
..     ...         ...          ...   ...       ...  ...
195      1           2            0     7         1    0
196      0           1            2     8         0    0
197      0           0            1     9         1    0
198      0           2            2     7         0    0
199      1           1            1     6         2    0

[200 rows x 6 columns]


,CustomerID,Books,Home Decor,Electronics,None,Clothing,NaN
0,C0001,1,1,3,6,0,0
1,C0002,0,2,0,7,2,0
2,C0003,0,2,1,7,1,0
3,C0004,3,3,2,3,0,0
4,C0005,0,1,2,8,0,0


In [10]:
new_final = final_data[['Books',  'Home Decor'  ,'Electronics'  ,'Clothing']]

In [11]:
new_final.head()

,Books,Home Decor,Electronics,Clothing
0,1,1,3,0
1,0,2,0,2
2,0,2,1,1
3,3,3,2,0
4,0,1,2,0


In [12]:
customer_ids = denormalized_data['CustomerID']

new_final.insert(0, 'CustomerID', customer_ids)

print(new_final)

    CustomerID  Books  Home Decor  Electronics  Clothing
0        C0001      1           1            3         0
1        C0001      0           2            0         2
2        C0001      0           2            1         1
3        C0001      3           3            2         0
4        C0001      0           1            2         0
..         ...    ...         ...          ...       ...
195      C0041      1           2            0         1
196      C0041      0           1            2         0
197      C0041      0           0            1         1
198      C0042      0           2            2         0
199      C0042      1           1            1         2

[200 rows x 5 columns]


In [13]:
customer_features = new_final[['Books',  'Home Decor',  'Electronics',  'Clothing']]
temp = cosine_similarity(customer_features)


In [14]:
customer_id_mapping = {f"C000{i+1}": i for i in range(temp.shape[0])}

index = pd.Index(customer_id_mapping.keys(), name="CustomerID")
columns = pd.Index(customer_id_mapping.keys(), name="CustomerID")
similarity_df = pd.DataFrame(temp, index=index, columns=columns)

In [15]:
similarity_df

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C00010,...,C000191,C000192,C000193,C000194,C000195,C000196,C000197,C000198,C000199,C000200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.213201,0.615457,0.771389,0.943880,0.369274,0.943880,0.770675,0.404520,0.095346,...,0.752618,0.861640,0.301511,0.836242,0.483494,0.369274,0.943880,0.639602,0.852803,0.569803
C0002,0.213201,1.000000,0.866025,0.452267,0.316228,0.577350,0.316228,0.774597,0.632456,0.670820,...,0.000000,0.288675,0.000000,0.588348,0.944911,0.866025,0.316228,0.500000,0.500000,0.801784
C0003,0.615457,0.866025,1.000000,0.696311,0.730297,0.500000,0.730297,0.968963,0.547723,0.387298,...,0.226455,0.500000,0.000000,0.792594,0.981981,0.833333,0.730297,0.577350,0.866025,0.771517
C0004,0.771389,0.452267,0.696311,1.000000,0.667424,0.783349,0.667424,0.817424,0.190693,0.202260,...,0.768706,0.609272,0.639602,0.946100,0.626783,0.783349,0.667424,0.301511,0.753778,0.644658
C0005,0.943880,0.316228,0.730297,0.667424,1.000000,0.182574,1.000000,0.816497,0.400000,0.000000,...,0.496139,0.730297,0.000000,0.744208,0.597614,0.365148,1.000000,0.632456,0.948683,0.507093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C000196,0.369274,0.866025,0.833333,0.783349,0.365148,0.833333,0.365148,0.819892,0.365148,0.516398,...,0.339683,0.333333,0.408248,0.792594,0.872872,1.000000,0.365148,0.288675,0.577350,0.771517
C000197,0.943880,0.316228,0.730297,0.667424,1.000000,0.182574,1.000000,0.816497,0.400000,0.000000,...,0.496139,0.730297,0.000000,0.744208,0.597614,0.365148,1.000000,0.632456,0.948683,0.507093
C000198,0.639602,0.500000,0.577350,0.301511,0.632456,0.288675,0.632456,0.645497,0.948683,0.670820,...,0.392232,0.866025,0.000000,0.588348,0.566947,0.288675,0.632456,1.000000,0.500000,0.801784


In [16]:

similarity_customer_ids = similarity_df.index.unique()

lookalikes = {}

for customer_id in similarity_customer_ids:

    scores = similarity_df.loc[customer_id].drop(customer_id)
    top_lookalikes = scores.nlargest(3).index.tolist()
    top_scores = scores.nlargest(3).values.tolist()
    lookalikes[customer_id] = list(zip(top_lookalikes, top_scores))


In [17]:
lookalike_df = pd.DataFrame.from_dict(lookalikes, orient='index')

lookalike_df = lookalike_df.stack().reset_index()
lookalike_df.columns = ['CustomerID', 'LookalikeIndex', 'LookalikeData']  # Rename columns

lookalike_df[['LookalikeID', 'SimilarityScore']] = pd.DataFrame(lookalike_df['LookalikeData'].tolist(), index=lookalike_df.index)

lookalike_df = lookalike_df.drop(columns=['LookalikeData', 'LookalikeIndex'])

lookalike_df.to_csv("Lookalike.csv", index=False)

In [18]:
customer_features

,Books,Home Decor,Electronics,Clothing
0,1,1,3,0
1,0,2,0,2
2,0,2,1,1
3,3,3,2,0
4,0,1,2,0
...,...,...,...,...
195,1,2,0,1
196,0,1,2,0
197,0,0,1,1
198,0,2,2,0
